In [1]:
import os 
from datetime import date
from matplotlib import pyplot as plt
import numpy as np

# we need to set the current path to the directory
# containing the suite3d repository, this hack should
# do the trick
os.chdir(os.path.dirname(os.path.abspath("")))

from suite3d.job import Job
from suite3d import io
from suite3d import plot_utils as plot

In [2]:
from suite3d import ui, plot_utils

In [3]:
# update this to point to the demo data! 
tifs = io.get_tif_paths('/mnt/md0/data/demo/lbm')[:10]
for tif in tifs: print(tif)

/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00001.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00002.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00003.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00004.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00005.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00006.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00007.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00008.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00009.tif
/mnt/md0/data/demo/lbm/2024-08-10_2_AH012_2P_00001_00010.tif


In [4]:
# update this to point to the demo data! 
tifs = io.get_tif_paths('/mnt/md0/data/demo/standard-2p')[:10]

In [13]:
# Set the mandatory parameters
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]) / 5,
    
    # planes to analyze. 0 is typically the flyback, so we exclude it here
    'planes' : np.array([1,2,3,4]), 
    # number of planes recorded by scanimage, including the flyback
    'n_ch_tif' : 5,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # which color channel is the functional one
     # voxel size in z,y,x in microns
    'voxel_size_um' : (20, 1.5, 1.5),

    # number of files to use for the initial pass
    # usually, ~500 frames is a good rule of thumb
    # we will just use 200 here for speed
    'n_init_files' :  2,

    # 3D GPU registration - fast! 
    '3d_reg' : True,
    'gpu_reg' : True,
    
    # note : 3D CPU is not supported yet
    'subtract_crosstalk' : False, # turn off some lbm-only features
    'fuse_strips' : False, # turn off some lbm-only features

    
}

In [14]:
# Create the job
job = Job(r'/mnt/md0/runs','demo-std', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

Job directory /mnt/md0/runs/s3d-demo-std already exists
Loading job directory for demo-std in /mnt/md0/runs
   Loading dirs 
      Found dir registered_fused_data
      Found dir summary
      Found dir iters
   Loading default params
      Updating param fs
      Updating param planes
      Updating param n_ch_tif
      Updating param tau
      Updating param lbm
      Updating param num_colors
      Updating param functional_color_channel
      Updating param voxel_size_um
      Updating param n_init_files
      Updating param 3d_reg
      Updating param gpu_reg
      Updating param subtract_crosstalk
      Updating param fuse_strips
   Updated main params file


In [7]:
import cupy

In [8]:
job.run_init_pass()

   Saved a copy of params at /mnt/md0/runs/s3d-demo-std/summary
   Updated main params file
Launching initial pass
Saving summary to /mnt/md0/runs/s3d-demo-std/summary/summary.npy
   Loading init tifs with 5 channels
      Loading tiff 1/2: /mnt/md0/data/demo/standard-2p/2023-04-12_701_ATL020_2P_00001_00004.tif
      Loading tiff 2/2: /mnt/md0/data/demo/standard-2p/2023-04-12_701_ATL020_2P_00001_00007.tif
   Loaded 2 files, total 1.56 GB
   Selecting 500 random frames from the init tif files
   Loaded movie with 500 frames and shape 4, 512, 512
      Enforcing positivity in mean image
   No crosstalk estimation or subtraction
   Using 3d registration
   Computing plane alignment shifts
   Applying plane alignment shifts
   Launching 3D GPU reference image calculation
      Seeding reference image with most active frames
      Completed iter 1 out of 8 in  12.39s using  69/500 frames
      Completed iter 2 out of 8 in  1.95s using  123/500 frames
      Completed iter 3 out of 8 in  2.07

In [9]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [10]:
# OPTIONAL: load and take a look at the reference image
summary = job.load_summary()
ref_img = summary['ref_img_3d']

# # view 1 plane at a time
# plot.show_img(ref_img[3], figsize=(3,4))

# # interactive 3D viewer
# plot.VolumeViewer(ref_img)


In [11]:
job.register()

      Found dir /mnt/md0/runs/s3d-demo-std/registered_fused_data
      Updating self.dirs tag registered_fused_data
   Saved a copy of params at /mnt/md0/runs/s3d-demo-std/registered_fused_data
   Updated main params file
   Starting registration: 3D: True, GPU: True
   Not clipping frames for registration
Will analyze 10 tifs in 10 batches
   Enforcing positivity
   Launching IO thread
         Memory at batch 0.  Total Used: 064.609 GB, Virtual Available: 451.993 GB, Virtual Used: 051.203 GB, Swap Used: 013.406 GB
Loading Batch 0 of 9
      Loading tiff 1/1: /mnt/md0/data/demo/standard-2p/2023-04-12_701_ATL020_2P_00001_00001.tif
   Loaded 1 files, total 0.78 GB
   Batch 0 IO thread joined
         Memory after IO thread joinTotal Used: 065.390 GB, Virtual Available: 451.211 GB, Virtual Used: 051.985 GB, Swap Used: 013.406 GB
         Memory after movie copied from threadTotal Used: 066.174 GB, Virtual Available: 450.428 GB, Virtual Used: 052.768 GB, Swap Used: 013.406 GB
         Mem

In [31]:
corr_map = job.calculate_corr_map()

In [27]:
res = job.load_corr_map_results()
vmap = res['vmap']

In [28]:
job.params['patch_size_xy'] = (550, 550)
# for speed, only segment a single patch
# for longer movies you should keep the smaller default setting
job.segment_rois()

      Found dir /mnt/md0/runs/s3d-demo-std/segmentation
      Updating self.dirs tag segmentation
   Saved a copy of params at /mnt/md0/runs/s3d-demo-std/segmentation
   Updated main params file
      Found dir /mnt/md0/runs/s3d-demo-std/rois
      Updating self.dirs tag rois
   Saving results to /mnt/md0/runs/s3d-demo-std/segmentation and /mnt/md0/runs/s3d-demo-std/rois 
dict_keys(['max_img', 'mean_img', 'vmap', 'vmap_raw', 'all_params'])
   Detecting from patch 1 / 1
      Found dir /mnt/md0/runs/s3d-demo-std/segmentation/patch-0000
         Loading 0.33 GB movie to memory, shape: (170, 4, 514, 513) 
         Loaded
         Loading movie patch to shared memory
         Loaded
      Starting extraction with peak_thresh: 0.100 and Th2: 5.000
         Iter 0000: running 16 ROIs in parallel
         Added cell 1 at 01, 358, 431, peak: 4.797, thresh: 5.000, 21 frames, 776 pixels
         Added cell 2 at 00, 259, 386, peak: 4.400, thresh: 5.000, 32 frames, 744 pixels
         Added cell 3

'/mnt/md0/runs/s3d-demo-std/rois'

In [29]:
job.compute_npil_masks()
traces = job.extract_and_deconvolve()

In [ ]:
job.export_results('path/to/output',result_dir_name='rois')

To take a look at the outputs in napari, navigate to the suite3d directory in a command shell and run the following:
```
python curation.py curation --output_dir /path/to/output/rois
```

If you want a quick visualization in the notebook, you can use the notebook volume visualizer:

In [18]:
seg_results = job.load_segmentation_results()

corrmap_vol = seg_results['info']['vmap']
mean_vol = seg_results['info']['mean_img']
cell_vol = ui.make_label_vols(seg_results['stats'],corrmap_vol.shape)

plot_utils.VolumeViewer(corrmap_vol, overlay=cell_vol[1]) 

      Loading from /mnt/md0/runs/s3d-demo-std/rois/info.npy
      Loading from /mnt/md0/runs/s3d-demo-std/rois/stats.npy
      Loading from /mnt/md0/runs/s3d-demo-std/rois/iscell.npy


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [11]:
# save the alpha channel of the RGBA volume above so all cells are the same color
ui.save_mrc('/mnt/md0/', 's3d-output.mrc', data=cell_vol[1][3], voxel_size=job.params['voxel_size_um'])

/mnt/md0/s3d-output.mrc
